### Import necessary libraries for data manipulation, visualization, and imputation.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.impute import KNNImputer

### Read training and test data, append a dummy target to test, combine both, and drop irrelevant columns.

In [2]:
df_train = pd.read_csv('train.csv')
df_test = pd.read_csv('test.csv')
df_test['Transported'] = False
df = pd.concat([df_train, df_test], sort = False)
df.drop(['Name', 'PassengerId'], axis = 1, inplace = True)
df.head()

,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Transported
0,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,False
1,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,True
2,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,False
3,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,False
4,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,True


### Ensure concatenation was correct and check for missing values.

In [3]:
df.shape[0] == df_train.shape[0] + df_test.shape[0]

True

In [4]:
df.isna().sum()

HomePlanet      288
CryoSleep       310
Cabin           299
Destination     274
Age             270
VIP             296
RoomService     263
FoodCourt       289
ShoppingMall    306
Spa             284
VRDeck          268
Transported       0
dtype: int64

### Extract structured features from the Cabin column and remove it.

In [5]:
df[['Deck', 'Num', 'Side']] = df['Cabin'].str.split('/', expand = True)
df = df.drop(columns = ['Cabin'])
df.head()

,HomePlanet,CryoSleep,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Transported,Deck,Num,Side
0,Europa,False,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,False,B,0,P
1,Earth,False,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,True,F,0,S
2,Europa,False,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,False,A,0,S
3,Europa,False,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,False,A,0,S
4,Earth,False,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,True,F,1,S


### Fill missing values in the newly created features with placeholders.

In [6]:
df['Deck'] = df['Deck'].fillna('U')
df['Num'] = df['Num'].fillna(-1)
df['Side'] = df['Side'].fillna('U')
df.isna().sum()

HomePlanet      288
CryoSleep       310
Destination     274
Age             270
VIP             296
RoomService     263
FoodCourt       289
ShoppingMall    306
Spa             284
VRDeck          268
Transported       0
Deck              0
Num               0
Side              0
dtype: int64

### Convert categorical text values to numeric format.

In [7]:
df['Deck'] = df['Deck'].map({'F' : 1, 'G' : 0, 'E' : 2, 'B': 3, 'C' : 4, 'D' : 5, 'A' : 6, 'U' : 7, 'T' : 8})
df['Side'] = df['Side'].map({'S': 2, 'P' : 1, 'U' : -1})

### Impute missing numerical values using K-Nearest Neighbors.

In [8]:
impute_list = ['Age', 'VIP', 'Num', 'CryoSleep', 'Side', 'Deck', 'RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck']
rest = list(set(df.columns) - set(impute_list))
df_rest = df[rest]
imp = KNNImputer()
df_imputed = imp.fit_transform(df[impute_list])
df_imputed = pd.DataFrame(df_imputed, columns = impute_list)
df = pd.concat([df_rest.reset_index(drop = True), df_imputed.reset_index(drop = True)], axis = 1)

In [9]:
df.isna().sum()

Transported       0
HomePlanet      288
Destination     274
Age               0
VIP               0
Num               0
CryoSleep         0
Side              0
Deck              0
RoomService       0
FoodCourt         0
ShoppingMall      0
Spa               0
VRDeck            0
dtype: int64

### Fill missing values in categorical columns and convert categorical variables into one-hot encoded format.

In [10]:
df['HomePlanet'] = df['HomePlanet'].fillna('U')
df['Destination'] = df['Destination'].fillna('U')
category_cols = ['HomePlanet', 'Destination']

for col in category_cols:
    df = pd.concat([df, pd.get_dummies(df[col], prefix = col)], axis = 1)

In [11]:
df.isna().sum()

Transported                  0
HomePlanet                   0
Destination                  0
Age                          0
VIP                          0
Num                          0
CryoSleep                    0
Side                         0
Deck                         0
RoomService                  0
FoodCourt                    0
ShoppingMall                 0
Spa                          0
VRDeck                       0
HomePlanet_Earth             0
HomePlanet_Europa            0
HomePlanet_Mars              0
HomePlanet_U                 0
Destination_55 Cancri e      0
Destination_PSO J318.5-22    0
Destination_TRAPPIST-1e      0
Destination_U                0
dtype: int64

In [12]:
df = df.drop(columns = category_cols)

### Create aggregate spending-related features.

In [13]:
bill_cols = ['RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck']
df['AmtSpent'] = df[bill_cols].sum(axis = 1)
df['StdAmtSpent'] = df[bill_cols].std(axis = 1)
df['MeanAmtSpent'] = df[bill_cols].mean(axis = 1)

In [14]:
df.corr()['Transported'].sort_values(ascending = False)

Transported                  1.000000
CryoSleep                    0.324525
HomePlanet_Europa            0.131977
Destination_55 Cancri e      0.083625
Side                         0.059872
Deck                         0.041775
FoodCourt                    0.034766
HomePlanet_U                 0.006403
HomePlanet_Mars              0.005643
ShoppingMall                 0.004189
Destination_PSO J318.5-22    0.000760
Destination_U               -0.000554
VIP                         -0.018569
Num                         -0.035240
Age                         -0.050592
Destination_TRAPPIST-1e     -0.072731
HomePlanet_Earth            -0.119644
StdAmtSpent                 -0.121134
AmtSpent                    -0.140416
MeanAmtSpent                -0.140416
VRDeck                      -0.142770
Spa                         -0.154816
RoomService                 -0.174750
Name: Transported, dtype: float64

### Build composite features from other meaningful attributes.

In [15]:
df['3_high_cols'] = df['CryoSleep'] + df['HomePlanet_Europa'] + df['Destination_55 Cancri e']
df['3_low_cols'] = df['AmtSpent'] + df['MeanAmtSpent'] + df['HomePlanet_Earth']

### Split the preprocessed combined dataset back into train and test sets.

In [16]:
df_train, df_test = df[:df_train.shape[0]], df[df_train.shape[0]:]
df_test = df_test.drop(columns = 'Transported')
df_train.shape, df_test.shape

((8693, 25), (4277, 24))

### Import classification models and evaluation tools.

In [17]:
from xgboost import XGBClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from lightgbm import LGBMClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

### Split data into training and validation sets and train and evaluate multiple models to compare performance.

In [18]:
X = df_train.drop(columns = 'Transported')
y = df_train['Transported']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

model_1 = XGBClassifier()
model_2 = DecisionTreeClassifier()
model_3 = RandomForestClassifier()
model_4 = LogisticRegression()
model_5 = LGBMClassifier()

In [19]:
model_1.fit(X_train, y_train)
pred = model_1.predict(X_test)
accuracy_score(y_test, pred)

0.7958596894767107

In [20]:
model_2.fit(X_train, y_train)
pred = model_2.predict(X_test)
accuracy_score(y_test, pred)

0.7452558941920644

In [21]:
model_3.fit(X_train, y_train)
pred = model_3.predict(X_test)
accuracy_score(y_test, pred)

0.7941345600920069

In [22]:
model_4.fit(X_train, y_train)
pred = model_4.predict(X_test)
accuracy_score(y_test, pred)

c:\Users\thomp\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.772857964347326

In [23]:
model_5.fit(X_train, y_train)
pred = model_5.predict(X_test)
accuracy_score(y_test, pred)

[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 3500, number of negative: 3454
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001121 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2703
[LightGBM] [Info] Number of data points in the train set: 6954, number of used features: 24
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.503307 -> initscore=0.013230
[LightGBM] [Info] Start training from score 0.013230


0.8004600345025877

### Make final predictions using the selected model (LightGBM) and prepare the submission file.

In [24]:
df_dummy = pd.read_csv('test.csv')

In [25]:
pred = model_5.predict(df_test)

final = pd.DataFrame()
final['PassengerID'] = df_dummy['PassengerId']
final['Transported'] = pred

final.to_csv('submission.csv', index = False)